In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import json
import pandas as pd
from pathlib import Path

pd.set_option("mode.chained_assignment", None)
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [ ]:
import nos

# Load all benchmarks within the .nos/benchmark directory
NOS_BENCHMARK_DIR = Path(nos.__file__).parent.parent / ".nos/benchmark"
print(f"Loading profiles in {NOS_BENCHMARK_DIR}")
benchmarks = list(Path(NOS_BENCHMARK_DIR).rglob("*.json"))
benchmarks

In [ ]:
df = pd.concat([pd.read_json(b) for b in benchmarks])
df

In [ ]:
from nos.common.profiler import NOS_PROFILE_DIR
from nos.common.profiler import Profiler

# Load all profiles within the cache/profile directory
print(f"Loading profiles in {NOS_PROFILE_DIR}")
profiles = list(Path(NOS_PROFILE_DIR).rglob("*.json"))

In [ ]:
# Load all profiles into a dataframe
def load_profile(filename):
    try:
        print(f"Loading {filename}")
        return Profiler.load_records(filename)
    except:
        print(f"Failed to load {filename}")
df = pd.concat([load_profile(p) for p in profiles])
print(f"Loaded profiles: {len(df)}")

# Rename columns names with profile_data.* 
df = df.rename(columns={col: col.replace("profile_data.", "") for col in df.columns})

# Convert memory columns to GB
for col in df.columns:
    if "::memory_" in col:
        df[col] /= 1024 **3

In [ ]:
df.T

In [ ]:
df = df.assign(
    latency_ms=lambda x: x["forward::execution.total_ms"] / x["forward::execution.num_iterations"],
    throughput=lambda x: x["forward::execution.num_iterations"] * x["batch_size"] / (x["forward::execution.total_ms"] / 1e3),
    gpu_utilization=lambda x: x["forward::execution.gpu_utilization"],
    cpu_utilization=lambda x: x["forward::execution.cpu_utilization"],
    cuda_torch_mem_usage_gb=lambda x: x["init::memory_gpu::post"],
    cuda_mem_usage_gb=lambda x: x["forward::memory_gpu::post"] - x["init::memory_gpu::pre"],
    model_id = lambda x: x["namespace"].astype(str) + " [b=" + x["batch_size"].astype(str) + ", d=" + x["device_index"].astype(str) + ", " + x["device_name"].astype(str) + "]"
)

In [ ]:
df.T

In [ ]:
from datetime import datetime
from pathlib import Path
from IPython.display import FileLink, FileLinks

date_str = datetime.utcnow().strftime("%Y%m%d")
path = Path.cwd() / f"profiles/nos_profile_aggregate_{date_str}.csv"
path.parent.mkdir(parents=True, exist_ok=True)

df.to_csv(str(path), index=False)
FileLink(path.relative_to(Path.cwd()))

In [ ]:
!pip install pygsheets

In [ ]:
import pygsheets

def write_to_gsheet(client_secret, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(client_secret=client_secret)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1

# Upload to nos-benchmarks
# Note: Download the client creds JSON from here: 
#    https://console.cloud.google.com/apis/credentials/oauthclient/851725070723-anse9hqc5clo0qm3lgtvshs5nnku8lhh.apps.googleusercontent.com?project=nos-benchmarks
WORKSHEET_ID = "14Ee2PR8c-1yQg6MCcElYymW67sjdSBlG-1AGW3Yuf2w"
date_str = datetime.utcnow().strftime("%Y%m%d")
write_to_gsheet("creds.json", WORKSHEET_ID, f"nos-benchmarks-{date_str}", df)
write_to_gsheet("creds.json", WORKSHEET_ID, f"nos-benchmarks", df)